Imports

In [1]:
# Imports
from pathlib import Path
import datetime
import numpy as np
import ROOT
from ROOT import TH2D, TH1D, TCanvas, gStyle
import uproot

from helpers import *

# ROOT interactive
# ROOT.gROOT.SetBatch(True)


Global settings for the analysis

In [2]:
# Set default style
setDefaultStyle()

# Globally defined dijet average pT ranges
dijet_pt_ave_ranges = [(60, 80), (80, 100), (100, 120), (120, 150), (150, 250), (250, 500)]


Output directory with the date

In [3]:
# Output directory based on date
today = datetime.date.today().strftime("%Y%m%d")
output_dir = Path("./") / today
output_dir.mkdir(parents=True, exist_ok=True)
print("Output PDF files will be stored in:", output_dir)


Output PDF files will be stored in: 20251113


In [ ]:
test_file = input_file(category="exp", direction="sum", eta_cut=19, jet_selection="jetId", systematics=None, trigger="MB")
print(f"Test file path: {test_file}")


Plot distributions in the laboratory and center of mass frames for the given experimental file

In [ ]:
show_plots = True

# Experimental files to read and process (check if exist automatically)
MB_file_name = input_file(category="exp", direction="sum", eta_cut=19, jet_selection="jetId", systematics=None, trigger="MB")
print(f"MB file path: {MB_file_name}")
# Jet60_file_name = input_file(category="exp", direction="sum", eta_cut=19, jet_selection="jetId", systematics=None, trigger="Jet60")
# Jet80_file_name = input_file(category="exp", direction="sum", eta_cut=19, jet_selection="jetId", systematics=None, trigger="Jet80")
# Jet100_file_name = input_file(category="exp", direction="sum", eta_cut=19, jet_selection="jetId", systematics=None, trigger="Jet100")

# List of input files
input_files_names = [
    MB_file_name
]

# Histogram names to process
histogram_names_to_read = [
    "hRecoDijetPtEtaWeighted",
    "hRecoDijetPtEtaCMWeighted",
    "hRecoDijetPtEtaCMForwardWeighted",
    "hRecoDijetPtEtaCMBackwardWeighted"
]

# Loop over input files and histogram names
for filename in input_files_names:
    # Extract trigger and direction from filename
    # filename = os.path.basename(filename)
    trigger = detect_trigger(str(filename))
    direction = detect_direction(str(filename))

    print(f"Processing: {filename}  |  Trigger: {trigger}  |  Direction: {direction}")

    # Create output directory for the trigger if it doesn't exist
    trigger_dir = os.path.join(output_dir, trigger)
    os.makedirs(trigger_dir, exist_ok=True)

    # Open the ROOT file with uproot
    with uproot.open(filename) as f:
        # Loop over histogram names
        for hist_name in histogram_names_to_read:

            hist = read_histogram(filename, hist_name)

            # if hist_name not in f:
            #     print(f"  Histogram {hist_name} not found in file: {filename}. Skipping.")
            #     continue

            # Retrieve histogram
            # hist = f[hist_name]
            # print(type(hist))
            
            # --- Choose η label for Y axis based on histogram name ---
            lname = hist_name.lower()
            if "etacm" in lname and "forward" not in lname and "backward" not in lname:
                y_title = r"$\eta_{\mathrm{CM}}^{\mathrm{dijet}}$"
                eta_tag = "dijetEtaCM_vs_pt"
            elif "etaweighted" in lname:
                y_title = r"$\eta^{\mathrm{dijet}}$"
                eta_tag = "dijetEtaLab_vs_pt"
            elif "forward" in lname:
                y_title = r"$\eta_{\mathrm{CM}}^{\mathrm{forward}}$"
                eta_tag = "dijetEtaForward_vs_pt"
            elif "backward" in lname:
                y_title = r"$\eta_{\mathrm{CM}}^{\mathrm{backward}}$"
                eta_tag = "dijetEtaBackward_vs_pt"
            else:
                y_title = "Y"
                eta_tag = "dijetEtaUnknown_vs_pt"

            out2d = os.path.join(
                trigger_dir,
                f"h{trigger}_{direction}_{eta_tag}.pdf"
            )

            plot_2d_histogram(h=hist, outname=out2d, xlabel=r"Dijet $p_{T}^{\mathrm{ave}}$ (GeV)", 
                              ylabel=y_title, show_plot=show_plots, is_preliminary=True, xlim=(40, 240) )



            # Loop over dijet pT average ranges
            # for (pt_low, pt_high) in dijet_pt_ave_ranges:
            #     print(f"    Dijet pT average range: {pt_low} - {pt_high} GeV")



                

In [ ]:
# Cell 3 — Load 2D histograms (TH2D) from ROOT file
hist_names = {
    "hRecoDijetPtEtaCMForwardWeighted",
    "hRecoDijetPtEtaCMBackwardWeighted"
}

histograms = load_histograms_with_tag(file_path, hist_names)
print("Histograms loaded successfully.")

h2_forward = histograms["hRecoDijetPtEtaCMForwardWeighted"]
h2_backward = histograms["hRecoDijetPtEtaCMBackwardWeighted"]



# def load_th2_from_root(file_path, hist_name):
#     """Return ROOT.TH2D cloned from file using uproot."""
#     with uproot.open(file_path) as f:
#         if hist_name not in f:
#             raise KeyError(f"Histogram {hist_name} not found in {file_path}")
#         vals, xedges, yedges = f[hist_name].to_numpy(flow=False)
    
#     # Convert to ROOT.TH2D
#     h2 = ROOT.TH2D(f"{hist_name}_{os.path.basename(file_path)}", hist_name,
#                 len(xedges)-1, np.array(xedges, dtype=np.float64),
#                 len(yedges)-1, np.array(yedges, dtype=np.float64))
#     set2DStyle(h2)

#     for ix in range(len(xedges)-1):
#         for iy in range(len(yedges)-1):
#             h2.SetBinContent(ix+1, iy+1, vals[ix, iy])
#     return h2

# Load both forward and backward histograms
# hist_names = {
#     "forward": "hRecoDijetPtEtaCMForwardWeighted",
#     "backward": "hRecoDijetPtEtaCMBackwardWeighted"
# }

# h2_forward = load_th2_from_root(file_path, hist_names["forward"])
# h2_backward = load_th2_from_root(file_path, hist_names["backward"])

# print("Histograms loaded successfully.")


In [ ]:
# Cell 4 — Project TH2D → TH1D for selected pT ranges and build ratio histograms
def plot_histograms_pdf(h_fw, h_bw, h_ratio, pt_low, pt_high, direction, output_dir):
    c = ROOT.TCanvas(f"c_{pt_low}_{pt_high}_{direction}", "", 600, 1200)
    c.Divide(1, 2)

    # Upper pad: forward/backward
    c.cd(1)
    setPadStyle()
    h_fw.Draw()
    h_bw.Draw("same")
    h_fw.SetTitle(f"{direction} | {pt_low}-{pt_high} GeV; #eta_{{CM}}; counts")
    h_fw.GetXaxis().SetRangeUser(0., 2.)
    ROOT.gPad.BuildLegend()


    # Lower pad: ratio
    c.cd(2)
    setPadStyle()
    h_ratio.Draw()
    h_ratio.SetTitle(f"{direction} Forward/Backward Ratio | {pt_low}-{pt_high} GeV; #eta_{{CM}}; Forward / Backward")
    h_ratio.GetYaxis().SetRangeUser(0.8, 1.2)
    h_ratio.GetXaxis().SetRangeUser(0., 2.)


    # Save as PDF
    pdf_file = output_dir / f"{direction}_pt_{pt_low}_{pt_high}.pdf"
    c.SaveAs(str(pdf_file))
    print("Saved PDF:", pdf_file)


In [ ]:
# Cell 5 — Create 1D projections and ratios



for pt_low, pt_high in pt_ranges:
    h_fw = projectDijetEtaFrom2D(h2_forward, pt_low, pt_high)    
    set1DStyle(h_fw, 0)
    h_bw = projectDijetEtaFrom2D(h2_backward, pt_low, pt_high)
    set1DStyle(h_bw, 1)
    h_ratio = make_fb_ratio(h_fw, h_bw)
    set1DStyle(h_ratio, 2)
    
    plot_histograms_pdf(h_fw, h_bw, h_ratio, pt_low, pt_high, direction_choice, output_dir)


In [4]:
#
# Plot JES and JER
#

embedding_file_name = input_file(category="embedding", direction="sum", eta_cut=19, jet_selection="jetId", systematics=None, trigger="MB")
print(f"Embedding file path: {embedding_file_name}")

# Open the ROOT file
f_embed = ROOT.TFile.Open(str(embedding_file_name))
if not f_embed or f_embed.IsZombie():
    raise RuntimeError(f"Failed to open ROOT file: {embedding_file_name}")

# NEW SUBDIRECTORY FOR JES & JER RESULTS
jesjer_dir = output_dir / "JESandJER"
jesjer_dir.mkdir(parents=True, exist_ok=True)
print("JES/JER plots will be stored in:", jesjer_dir)

hist_names_jes_jer = [
    "hInclusiveJetJESGenPtGenEtaPtHatWeighted",
    "hLeadJetJESGenPtEtaPtHatWeighted",
    "hSubLeadJetJESGenPtEtaPtHatWeighted"
]

# Define eta and pT slices for projections
eta_slices = [ (-3.0, -2.4), (-2.4, -1.6), (-1.6, -0.8), (-0.8, 0.0),
               (0.0, 0.8), (0.8, 1.6), (1.6, 2.4), (2.4, 3.0) ]

pt_slices = [ (40, 60), (60, 100), (100, 120), (120, 180), (180, 250), (250, 1000) ]

# -------------------------------------------------------
# PROCESS EACH SPARSE HISTOGRAM
# -------------------------------------------------------

for hname in hist_names_jes_jer:

    sparse = f_embed.Get(hname)
    if not sparse:
        print(f"[ERROR] histogram {hname} not found")
        continue

    prefix = jes_prefix_from_name(hname)   
    print(f"\n=== Processing {hname} ===")


    #
    # ---------------------------------------------------
    # CASE A: JES/JER vs pT for slices in eta
    # ---------------------------------------------------
    #
    for (eta_low, eta_hi) in eta_slices:

        print(f"  -> Eta slice: {eta_low} to {eta_hi}")

        # projection pT_reco/pT_ref vs pT_ref
        lo_s, hi_s = fmt_range(eta_low, eta_hi)
        h2 = project_sparse(
            sparse,
            axes=(0, 1),      # (pT_reco/pT_ref, pT_ref)
            ranges={2: (eta_low, eta_hi)},
            name=f"{prefix}_jes_eta_{lo_s}_{hi_s}"
        )

        h2.GetXaxis().SetTitle("p_{T}^{ref} (GeV)")
        h2.GetYaxis().SetTitle("p_{T}^{reco}/p_{T}^{ref}")

        # FitSlicesY → JES & JER
        hJES, hJER = extract_jes_jer(h2)

        # plot individually
        c = TCanvas("c","c",1500,500)
        c.Divide(3,1)

        c.cd(1); setPadStyle(); set2DStyle(h2); h2.Draw("colz"); h2.GetXaxis().SetRangeUser(30, 300)
        c.cd(2); setPadStyle(); set1DStyle(hJES, 2); hJES.Draw("P"); hJES.GetXaxis().SetTitle("p_{T}^{ref} (GeV)"); hJES.GetYaxis().SetRangeUser(0.8, 1.2); hJES.GetXaxis().SetRangeUser(30, 300)
        eta_label = f"{eta_low} < #eta < {eta_hi}"
        draw_slice_label(prefix, eta_label)
        c.cd(3); setPadStyle(); set1DStyle(hJER, 2); hJER.Draw("P"); hJER.GetXaxis().SetTitle("p_{T}^{ref} (GeV)"); hJER.GetYaxis().SetRangeUser(0.0, 0.3); hJER.GetXaxis().SetRangeUser(30, 300)

        outname = jesjer_dir / f"{h2.GetName()}.pdf"
        c.SaveAs(str(outname))
        del c

    #   
    # ---------------------------------------------------
    # CASE B: JES/JER vs eta for slices in pT
    # ---------------------------------------------------
    #
    for (pt_low, pt_hi) in pt_slices:

        print(f"  -> pT slice: {pt_low} to {pt_hi}")

        lo_s, hi_s = fmt_range(pt_low, pt_hi)
        h2 = project_sparse(
            sparse,
            axes=(0, 2),      # (pT_reco/pT_ref, eta_ref)
            ranges={1: (pt_low, pt_hi)},
            name=f"{prefix}_jes_pt_{lo_s}_{hi_s}"
        )

        h2.GetXaxis().SetTitle("#eta_{ref}")
        h2.GetYaxis().SetTitle("p_{T}^{reco}/p_{T}^{ref}")

        # FitSlicesY → JES & JER
        hJES, hJER = extract_jes_jer(h2)

        # plot individually
        c = TCanvas("c","c",1500,500)
        c.Divide(3,1)

        c.cd(1); setPadStyle(); set2DStyle(h2); h2.Draw("colz"); h2.GetXaxis().SetRangeUser(-3, 3)
        c.cd(2); setPadStyle(); set1DStyle(hJES, 2); hJES.Draw("P");  hJES.GetXaxis().SetTitle("#eta_{ref}"); hJES.GetYaxis().SetRangeUser(0.8, 1.2); hJES.GetXaxis().SetRangeUser(-3, 3)
        pt_label = f"{pt_low} < p_{{T}}^{{ref}} < {pt_hi} GeV"
        draw_slice_label(prefix, pt_label)
        c.cd(3); setPadStyle(); set1DStyle(hJER, 2); hJER.Draw("P"); hJER.GetXaxis().SetTitle("#eta_{ref}"); hJER.GetYaxis().SetRangeUser(0.0, 0.3); hJER.GetXaxis().SetRangeUser(-3, 3)

        outname = jesjer_dir / f"{h2.GetName()}.pdf"
        c.SaveAs(str(outname))
        del c



Embedding file path: /Users/gnigmat/cernbox/ana/pPb8160/embedding/oEmbedding_pPb8160_def_ak4_jetId_eta19.root
JES/JER plots will be stored in: 20251113/JESandJER

=== Processing hInclusiveJetJESGenPtGenEtaPtHatWeighted ===
  -> Eta slice: -3.0 to -2.4
  -> Eta slice: -2.4 to -1.6
  -> Eta slice: -1.6 to -0.8
  -> Eta slice: -0.8 to 0.0
  -> Eta slice: 0.0 to 0.8
  -> Eta slice: 0.8 to 1.6
  -> Eta slice: 1.6 to 2.4
  -> Eta slice: 2.4 to 3.0
  -> pT slice: 40 to 60
  -> pT slice: 60 to 100
  -> pT slice: 100 to 120
  -> pT slice: 120 to 180
  -> pT slice: 180 to 250
  -> pT slice: 250 to 1000

=== Processing hLeadJetJESGenPtEtaPtHatWeighted ===
  -> Eta slice: -3.0 to -2.4
  -> Eta slice: -2.4 to -1.6
  -> Eta slice: -1.6 to -0.8
  -> Eta slice: -0.8 to 0.0
  -> Eta slice: 0.0 to 0.8
  -> Eta slice: 0.8 to 1.6
  -> Eta slice: 1.6 to 2.4
  -> Eta slice: 2.4 to 3.0
  -> pT slice: 40 to 60
  -> pT slice: 60 to 100
  -> pT slice: 100 to 120
  -> pT slice: 120 to 180
  -> pT slice: 180 to 2

Info in <TCanvas::Print>: pdf file 20251113/JESandJER/hInclusive_jes_eta_m3_0_m2_4.pdf has been created
Info in <TCanvas::Print>: pdf file 20251113/JESandJER/hInclusive_jes_eta_m2_4_m1_6.pdf has been created
Info in <TCanvas::Print>: pdf file 20251113/JESandJER/hInclusive_jes_eta_m1_6_m0_8.pdf has been created
Info in <TCanvas::Print>: pdf file 20251113/JESandJER/hInclusive_jes_eta_m0_8_0_0.pdf has been created
Info in <TCanvas::Print>: pdf file 20251113/JESandJER/hInclusive_jes_eta_0_0_0_8.pdf has been created
Info in <TCanvas::Print>: pdf file 20251113/JESandJER/hInclusive_jes_eta_0_8_1_6.pdf has been created
Info in <TCanvas::Print>: pdf file 20251113/JESandJER/hInclusive_jes_eta_1_6_2_4.pdf has been created
Info in <TCanvas::Print>: pdf file 20251113/JESandJER/hInclusive_jes_eta_2_4_3_0.pdf has been created
Info in <TCanvas::Print>: pdf file 20251113/JESandJER/hInclusive_jes_pt_40_60.pdf has been created
Info in <TCanvas::Print>: pdf file 20251113/JESandJER/hInclusive_jes_pt_60_100

In [ ]:

MB_file_name = "~/cernbox/ana/pPb8160/exp/MB_pPb8160_PD_ak4_jetId_eta19.root"



# histograms to read
hist_names = [
    "hRecoDijetPtEtaWeighted",
    "hRecoDijetPtEtaCMWeighted",
    "hRecoDijetPtEtaCMForwardWeighted",
    "hRecoDijetPtEtaCMBackwardWeighted",
]

# pTave intervals in GeV
pt_bins = [
    (50, 60),
    (60, 80),
    (80, 100),
    (100, 120),
]

# Output file for projections
out_file = ROOT.TFile("dijetEta_projections.root", "RECREATE")

#-----------------------------------------
# Open ROOT file
#-----------------------------------------
f = ROOT.TFile.Open(MB_file_name)
if not f or f.IsZombie():
    raise IOError(f"Could not open file: {MB_file_name}")

#-----------------------------------------
# Helper: project y-axis for given pTave range
#-----------------------------------------
def project_y(h2, pt_low, pt_high):
    """
    Project TH2 histogram onto y-axis for a given pT_ave range.
    """
    xaxis = h2.GetXaxis()

    # Convert GeV values to bin numbers
    bin_low  = xaxis.FindBin(pt_low)
    bin_high = xaxis.FindBin(pt_high)

    # Safety: ROOT allows low>high; it auto-swaps
    proj_name = f"{h2.GetName()}_pt_{pt_low}_{pt_high}"
    proj = h2.ProjectionY(proj_name, bin_low, bin_high)

    # Detach from file so hist is not deleted when file closes
    proj.SetDirectory(0)
    proj.SetTitle(f"{h2.GetTitle()} (pTave {pt_low}-{pt_high} GeV)")

    return proj

#---------------------------------------------------------
# Storage for overlay + ratios
#---------------------------------------------------------
projections_all = {}
ratios_all = {}     # <--- NEW

#---------------------------------------------------------
# Main projection loop
#---------------------------------------------------------
for hname in hist_names:
    h2 = f.Get(hname)
    if not h2 or not h2.InheritsFrom("TH2"):
        print(f"Skipping {hname}")
        continue

    print(f"Processing histogram: {hname}")
    out_file.cd()

    projections = []

    for (pt_low, pt_high) in pt_bins:
        proj = project_y(h2, pt_low, pt_high)
        proj.Write()
        projections.append(proj)

    projections_all[hname] = projections


#---------------------------------------------------------
# Compute Forward/Backward ratios for each pT bin
#---------------------------------------------------------
forward_name  = "hRecoDijetPtEtaCMForwardWeighted"
backward_name = "hRecoDijetPtEtaCMBackwardWeighted"

if forward_name in projections_all and backward_name in projections_all:
    print("\nComputing F/B ratios...")

    ratios_list = []

    for idx, (pt_low, pt_high) in enumerate(pt_bins):
        F = projections_all[forward_name][idx]
        B = projections_all[backward_name][idx]

        ratio_name = f"hEtaFB_ratio_pt_{pt_low}_{pt_high}"
        R = F.Clone(ratio_name)
        R.SetDirectory(0)
        R.SetTitle(f"Forward / Backward (pTave {pt_low}-{pt_high} GeV)")

        R.Divide(B)

        out_file.cd()
        R.Write()

        ratios_list.append(R)

    ratios_all["FoverB"] = ratios_list
else:
    print("Forward or backward histogram not found — cannot compute ratios.")

#-----------------------------------------
# Finish
#-----------------------------------------
out_file.Close()
f.Close()

print("Done. Projections written to dijetEta_projections.root")
